# Observatory Distances

In this notebook, a dataframe is built based on the distances from each observatory to the Apophis in every time step.

## Lo de Google Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!ls "/gdrive/MyDrive/Apophis/Data"

 AllRadiotelescopes.xlsx  'Radio coverage Arecibo Telescope.png'
 ephemeris2.xlsx	  'Radio coverage FAST.png'
 ephemeris3.xlsx	  'Radio coverage Goldstone Radio Telescope.png'
 ephemeris.xlsx		   utils.ipynb


In [3]:
%cd "/gdrive/MyDrive/Apophis/Data"

/gdrive/MyDrive/Apophis/Data


In [4]:
import pandas as pd
import pylab as plt
import numpy as np

In [5]:
ephemeris = pd.read_excel("ephemeris.xlsx")
radio = pd.read_excel("AllRadiotelescopes.xlsx")
print(radio)

                                             Name  ...                                               Link
0                               Arecibo Telescope  ...                                                NaN
1                       Goldstone Radio Telescope  ...  https://en.wikipedia.org/wiki/Goldstone_Deep_S...
2                 Green Bank Interferometer (GBI)  ...                                                NaN
3                      Green Bank Telescope (GBT)  ...                                                NaN
4         Green Bank 140 Foot Telescope (140foot)  ...                                                NaN
5                          Very Large Array (VLA)  ...                                                NaN
6                 Very Long Baseline Array (VLBA)  ...                                                NaN
7                                            FAST  ...                                                NaN
8                                           CD

## Distances

In [17]:
"ASTROPY"
from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun
from astropy.constants import c
import astropy.units as u

In [18]:
import warnings
warnings.simplefilter('ignore', UserWarning)

In [35]:
# Obtains the Astropy Position Vector of given earth coordinates
pos = lambda x,h: EarthLocation(lon=x['Longitude']*u.deg,lat=x['Latitude']*u.deg,height=h*u.earthRad)

# Transforms an Astropy object into a simple 3-tuple vector
vec = lambda a:  (a.x.value,
                  a.y.value,
                  a.z.value)

# Obtains the difference between two Astropy Vectors
vecdiff = lambda a,b: (a.x.value-b.x.value,
                       a.y.value-b.y.value,
                       a.z.value-b.z.value)

# Returns the magnitude of a vector
dist = lambda v: np.linalg.norm(v)


distances = pd.DataFrame(columns=['datetime_jd','datetime_str'])
distances[['datetime_jd','datetime_str']] = ephemeris[['datetime_jd','datetime_str']]

def distance_to_apophis(obs,row):
    pos1 = pos(obs,0)
    posA = pos(row,row['delta (Rt)'])
    vec1 = vecdiff(posA,pos1) # Vector from obs1 to Apophis
    dist1 = dist(vec1)*u.earthRad # Magnitudes
    return dist1.value

for i,obs in radio.iterrows():
    name = obs['Name']
    obs_distances = []
    for j,row in ephemeris.iterrows():
            obs_distances += [distance_to_apophis(obs,row)]
    distances[name] = obs_distances
distances

,datetime_jd,datetime_str,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,2.462240e+06,2029-Apr-13 13:30,31.939220,31.381215,31.831009,31.831004,31.831041,31.494291,31.494284,30.810662,30.191358,30.858494,30.702620,30.710129,30.710131,30.632213,30.666222,30.689203,30.710202,30.710202,30.675374
1,2.462240e+06,2029-Apr-13 13:35,31.670129,31.115787,31.561720,31.561715,31.561750,31.229028,31.229021,30.512405,29.907011,30.568536,30.414375,30.421358,30.421356,30.339626,30.374870,30.400958,30.419906,30.419906,30.384092
2,2.462240e+06,2029-Apr-13 13:40,31.400701,30.850400,31.292218,31.292214,31.292247,30.963736,30.963729,30.214417,29.623116,30.278902,30.126525,30.132976,30.132971,30.047431,30.083908,30.113113,30.129992,30.129992,30.093197
3,2.462240e+06,2029-Apr-13 13:45,31.130929,30.585047,31.022497,31.022493,31.022525,30.698409,30.698402,29.916710,29.339680,29.989598,29.839077,29.844991,29.844981,29.755638,29.793344,29.825675,29.840466,29.840466,29.802698
4,2.462240e+06,2029-Apr-13 13:50,30.860806,30.319719,30.752550,30.752546,30.752576,30.433036,30.433029,29.619294,29.056708,29.700629,29.552037,29.557408,29.557395,29.464256,29.503187,29.538650,29.551335,29.551335,29.512602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2.462241e+06,2029-Apr-14 05:45,30.995155,30.727928,30.770941,30.770937,30.770858,30.796666,30.796662,29.336876,30.156564,29.519993,29.576687,29.559813,29.559698,29.465540,29.490830,29.577840,29.514815,29.514815,29.491452
196,2.462241e+06,2029-Apr-14 05:50,31.271144,31.019997,31.052164,31.052160,31.052081,31.086236,31.086232,29.624857,30.453287,29.813512,29.873245,29.855984,29.855866,29.760173,29.785938,29.874559,29.810082,29.810082,29.786503
197,2.462241e+06,2029-Apr-14 05:55,31.546677,31.311678,31.332978,31.332974,31.332894,31.375388,31.375385,29.913314,30.750251,30.107315,30.170074,30.152436,30.152316,30.055184,30.081393,30.171551,30.105666,30.105666,30.081898
198,2.462241e+06,2029-Apr-14 06:00,31.821758,31.602968,31.613385,31.613381,31.613300,31.664120,31.664117,30.202240,31.047444,30.401392,30.467163,30.449158,30.449037,30.350566,30.377185,30.468804,30.401555,30.401555,30.377626


In [36]:
distances.to_csv("observatory_distances_indexes.csv")